# Notebook Tarea 2 Sistemas distribuidos

### Importación de librerías

In [15]:
import lithops
import pandas as pd
from io import StringIO
from lithops import FunctionExecutor
from pylab import *
import matplotlib.pyplot as plt
import sqlalchemy
from pandasql import sqldf
import pandasql
from lithops import Storage
import os
from dotenv import load_dotenv

### Variables globales

In [16]:
fexec = lithops.FunctionExecutor(runtime = 'enzeax/dockerfile:0.4',runtime_memory=2048)

2021-06-29 17:59:05,581 [INFO] lithops.config -- Lithops v2.3.4
2021-06-29 17:59:05,592 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-de
2021-06-29 17:59:05,595 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: eu-de - Namespace: Namespace-1441991
2021-06-29 17:59:05,596 [INFO] lithops.executors -- Serverless Executor created with ID: f6c649-2


### Funciones auxiliares

In [17]:
def formateo(pandaQuery):
    pandaQuery = pandaQuery.str.replace('00:00:00.000000','',regex=True)
    pandaQuery = pandaQuery.str.replace('2020','20',regex=True)
    pandaQuery = pandaQuery.str.replace('2021','21',regex=True)
    return pandaQuery

In [18]:
def grafico(pandaQuery, x, y):
    fig, ax = plt.subplots(figsize=(16, 7))
    pandaQuery[x] = formateo(pandaQuery[x])
    ax.plot(pandaQuery[x], pandaQuery[y])
    plt.show()

In [19]:
def getDataInfo(selectQuery):
    storage = Storage()
    data = storage.get_object('test-bucket-1441991', 'database.csv')

    format_data = str(data[0:-1], 'utf-8')
    database = pd.read_csv(StringIO(format_data))

    database["TipusCasData"]= pd.to_datetime(database["TipusCasData"])
    database = database.sort_values(by="TipusCasData")
    query = sqldf(selectQuery)

    return query

In [20]:
def graph_plot_multiline(query1, x1, y1, label1, query2, x2, y2, label2):
    fig, ax = plt.subplots(figsize=(16, 7))
    query1[x1] = formateo(query1[x1])
    ax.plot(query1[x1], query1[y1], label = label1)
    query2[x2] = formateo(query2[x2])
    ax.plot(query2[x2], query2[y2], label = label2)
    plt.legend()
    plt.show()

### Query 1: consulta n casos por tiempo en la comarca del Tarragones

In [21]:
fexec.call_async(getDataInfo, "SELECT NumCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' GROUP BY TipusCasData", runtime_memory=2048)
query = fexec.get_result()

2021-06-29 17:59:05,619 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A000 - Selected Runtime: enzeax/dockerfile:0.4 - 2048MB
2021-06-29 17:59:05,622 [INFO] lithops.job.job -- ExecutorID f6c649-2 | JobID A000 - Uploading function and data - Total: 1.1KiB
2021-06-29 17:59:06,208 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A000 - Starting function invocation: getDataInfo() - Total: 1 activations
2021-06-29 17:59:06,209 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A000 - View execution logs at /tmp/lithops/logs/f6c649-2-A000.log
2021-06-29 17:59:06,211 [INFO] lithops.wait -- ExecutorID f6c649-2 - Getting results from functions


2021-06-29 17:59:17,350 [INFO] lithops.executors -- ExecutorID f6c649-2 - Cleaning temporary data


In [22]:
query['TipusCasData'] = formateo(query['TipusCasData'])
grafico(query, 'TipusCasData', 'NumCasos')

### Query 2: consulta n. casos por comarca A-L

In [23]:
fexec.call_async(getDataInfo, "SELECT SUM(NumCasos) AS TotalCasos, ComarcaDescripcio FROM database WHERE ComarcaDescripcio < 'L%'  GROUP BY ComarcaDescripcio", runtime_memory=2048)
query = fexec.get_result()

2021-06-29 17:59:18,845 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A001 - Selected Runtime: enzeax/dockerfile:0.4 - 2048MB
2021-06-29 17:59:18,849 [INFO] lithops.job.job -- ExecutorID f6c649-2 | JobID A001 - Uploading function and data - Total: 1.1KiB
2021-06-29 17:59:19,180 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A001 - Starting function invocation: getDataInfo() - Total: 1 activations
2021-06-29 17:59:19,181 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A001 - View execution logs at /tmp/lithops/logs/f6c649-2-A001.log
2021-06-29 17:59:19,183 [INFO] lithops.wait -- ExecutorID f6c649-2 - Getting results from functions


2021-06-29 17:59:30,268 [INFO] lithops.executors -- ExecutorID f6c649-2 - Cleaning temporary data


In [24]:
fig, ax = plt.subplots(figsize=(16, 7))
ax.barh(query['ComarcaDescripcio'], query['TotalCasos'])
plt.show()

### Query 3: consulta n. casos por comarca M-Z

In [25]:
fexec.call_async(getDataInfo, "SELECT SUM(NumCasos) AS TotalCasos, ComarcaDescripcio FROM database WHERE ComarcaDescripcio >= 'L%'  GROUP BY ComarcaDescripcio", runtime_memory=2048)
query = fexec.get_result()

2021-06-29 17:59:31,446 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A002 - Selected Runtime: enzeax/dockerfile:0.4 - 2048MB
2021-06-29 17:59:31,449 [INFO] lithops.job.job -- ExecutorID f6c649-2 | JobID A002 - Uploading function and data - Total: 1.1KiB
2021-06-29 17:59:31,686 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A002 - Starting function invocation: getDataInfo() - Total: 1 activations
2021-06-29 17:59:31,687 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A002 - View execution logs at /tmp/lithops/logs/f6c649-2-A002.log
2021-06-29 17:59:31,689 [INFO] lithops.wait -- ExecutorID f6c649-2 - Getting results from functions


2021-06-29 17:59:42,771 [INFO] lithops.executors -- ExecutorID f6c649-2 - Cleaning temporary data


In [26]:
fig, ax = plt.subplots(figsize=(16, 7))
ax.barh(query['ComarcaDescripcio'], query['TotalCasos'])
plt.show()

### Query 4: consulta n. casos en un mes de todas las comarcas

In [27]:
fexec.call_async(getDataInfo, "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE TipusCasData BETWEEN '2020-01-01' AND '2020-02-01' GROUP BY TipusCasData", runtime_memory=2048)
query = fexec.get_result()

2021-06-29 17:59:43,886 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A003 - Selected Runtime: enzeax/dockerfile:0.4 - 2048MB
2021-06-29 17:59:43,888 [INFO] lithops.job.job -- ExecutorID f6c649-2 | JobID A003 - Uploading function and data - Total: 1.1KiB
2021-06-29 17:59:44,115 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A003 - Starting function invocation: getDataInfo() - Total: 1 activations
2021-06-29 17:59:44,116 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID A003 - View execution logs at /tmp/lithops/logs/f6c649-2-A003.log
2021-06-29 17:59:44,119 [INFO] lithops.wait -- ExecutorID f6c649-2 - Getting results from functions


2021-06-29 17:59:55,201 [INFO] lithops.executors -- ExecutorID f6c649-2 - Cleaning temporary data


In [28]:
query['TipusCasData'] = formateo(query['TipusCasData'])
grafico(query, 'TipusCasData', 'TotalCasos')

### Query 5: consulta paralelizadas de n. casos de la comarca del Tarragones

In [29]:
fexec.map(getDataInfo, [
        "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' AND TipusCasData BETWEEN '2020-01-01' AND '2020-03-01' GROUP BY TipusCasData",
        "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' AND TipusCasData BETWEEN '2020-03-01' AND '2020-05-01' GROUP BY TipusCasData",
        "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' AND TipusCasData BETWEEN '2020-05-01' AND '2020-07-01' GROUP BY TipusCasData",
        "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' AND TipusCasData BETWEEN '2020-07-01' AND '2020-09-01' GROUP BY TipusCasData",
        "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' AND TipusCasData BETWEEN '2020-09-01' AND '2020-11-01' GROUP BY TipusCasData",
        "SELECT SUM(NumCasos) AS TotalCasos, TipusCasData FROM database WHERE ComarcaDescripcio='TARRAGONES' AND TipusCasData BETWEEN '2020-11-01' AND '2021-01-01' GROUP BY TipusCasData"
    ])
query = fexec.get_result()

2021-06-29 17:59:56,271 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID M004 - Selected Runtime: enzeax/dockerfile:0.4 - 2048MB
2021-06-29 17:59:56,275 [INFO] lithops.job.job -- ExecutorID f6c649-2 | JobID M004 - Uploading function and data - Total: 2.1KiB
2021-06-29 17:59:56,459 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID M004 - Starting function invocation: getDataInfo() - Total: 6 activations
2021-06-29 17:59:56,463 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID M004 - View execution logs at /tmp/lithops/logs/f6c649-2-M004.log
2021-06-29 17:59:56,466 [INFO] lithops.wait -- ExecutorID f6c649-2 - Getting results from functions


2021-06-29 18:00:31,710 [INFO] lithops.executors -- ExecutorID f6c649-2 - Cleaning temporary data


In [30]:
for q in query:
        grafico(q, 'TipusCasData', 'TotalCasos')

### Query 6: consulta número de alta diarias y defunciones diarias durante el año 2020

In [31]:
fexec.map(getDataInfo, [
    "SELECT DISTINCT TipusCasData, AltesDiaries FROM database WHERE AltesDiaries IS NOT NULL",
    "SELECT DISTINCT TipusCasData, DefuncionsDiaries FROM database WHERE AltesDiaries IS NOT NULL"
])
query = fexec.get_result()

2021-06-29 18:00:38,509 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID M005 - Selected Runtime: enzeax/dockerfile:0.4 - 2048MB
2021-06-29 18:00:38,513 [INFO] lithops.job.job -- ExecutorID f6c649-2 | JobID M005 - Uploading function and data - Total: 1.2KiB
2021-06-29 18:00:38,834 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID M005 - Starting function invocation: getDataInfo() - Total: 2 activations
2021-06-29 18:00:38,837 [INFO] lithops.invokers -- ExecutorID f6c649-2 | JobID M005 - View execution logs at /tmp/lithops/logs/f6c649-2-M005.log
2021-06-29 18:00:38,839 [INFO] lithops.wait -- ExecutorID f6c649-2 - Getting results from functions


2021-06-29 18:00:48,926 [INFO] lithops.executors -- ExecutorID f6c649-2 - Cleaning temporary data


In [32]:
graph_plot_multiline(query[0], 'TipusCasData', 'AltesDiaries', 'Altes diàries', query[1], 'TipusCasData', 'DefuncionsDiaries', 'Defuncions diàries')